# MAS for Financial Analysis

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from dotenv import load_dotenv

dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))

load_dotenv(dotenv_path)

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Tasks

In [ ]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
    "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
    "uses statistical modeling and machine learning "
    "to provide crucial insights. With a knack for data, "
    "the Data Analyst Agent is the cornerstone for "
    "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [ ]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
    "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
    "markets and quantitative analysis, this agent "
    "devises and refines trading strategies. It evaluates "
    "the performance of different approaches to determine "
    "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [ ]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
    "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
    "and logistical details of potential trades. By evaluating "
    "these factors, it provides well-founded suggestions for "
    "when and how trades should be executed to maximize "
    "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

In [ ]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
    "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
    "and market dynamics, this agent scrutinizes the potential "
    "risks of proposed trades. It offers a detailed analysis of "
    "risk exposure and suggests safeguards to ensure that "
    "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
)

## Agents

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Crew

In [ ]:
from crewai import Crew, Process, LLM

financial_trading_crew = Crew(
    agents=[
        data_analyst_agent,
        trading_strategy_agent,
        execution_agent,
        risk_management_agent,
    ],
    tasks=[
        data_analysis_task,
        strategy_development_task,
        execution_planning_task,
        risk_assessment_task,
    ],
    manager_llm=LLM(model="openai/gpt-4o", temperature=0),  # delegates the work
    process=Process.hierarchical,
    verbose=True,
)

In [ ]:
financial_trading_inputs = {
    "stock_selection": "AAPL",
    "initial_capital": "100000",
    "risk_tolerance": "Medium",
    "trading_strategy_preference": "Day Trading",
    "news_impact_consideration": True,
}

In [ ]:
results = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 30d7a8b4-3e5f-4a7f-bfc2-8e846198ad84                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling     │
│  and machine learning to identify trends and predict market movements.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Thought: Thought: To monitor and analyze market data for AAPL, I need to gather the latest market data and     │
│  trends. I will start by searching the internet for the latest news and data on AAPL to understand the current  │
│  market situation.                                                                                              │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"AAPL stock market news October 2023\"}"                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"question\": \"Can you provide insights from the AAPL historical stock data analysis, focusing on           │
│  statistical modeling and machine learning results? This will help in understanding the data trends and         │
│  patterns for developing trading strategies.\", \"context\": \"The user prefers day trading with a medium risk  │
│  tolerance. The analysis plan provides a comprehensive approach to analyzing AAPL historical stock data using   │
│  statistical modeling and machine learning. It includes data preparation, feature engineering, modeling         │
│  approaches, and backtesting strategies.\", \"coworker\": \"Data Analyst\"}"                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Here is a comprehensive, coworker-ready briefing on insights from AAPL historical stock data using             │
│  statistical models and machine learning, tailored for day trading with a medium risk tolerance. The content    │
│  ties together established findings from the literature and practical implications for a live trading           │
│  workflow.                                                                                                      │
│                                                                                                                 │
│  Executive summary                                                                                              │
│  - Across multiple studies, Bi-LSTM and its variants consistently outperform traditional statistical models     │
│  (ARIMA, GARCH) for predicting daily closing prices on liquid equities like AAPL, especially when nonlinear     │
│  patterns and longer temporal dependencies drive price moves.                                                   │
│  - ARIMA and GARCH remain valuable for understanding linear trends and volatility dynamics, but their ability   │
│  to forecast price levels is generally weaker than deep learning approaches in volatile, high-signal equities.  │
│  - Hybrid approaches (ARIMA-LSTM, LSTM-GARCH or Bi-LSTM with volatility awareness) can provide incremental      │
│  gains by combining strengths of linear trend captures with nonlinear pattern recognition and volatility risk   │
│  signals.                                                                                                       │
│  - Backtesting evidence suggests that signals derived from Bi-LSTM or hybrid models can improve risk-adjusted   │
│  returns relative to naive benchmarks when appropriately regularized (rolling training, walk-forward            │
│  validation, position sizing, and stop rules). However, gains are not universal across regimes, and             │
│  overfitting remains a key risk.                                                                                │
│                                                                                                                 │
│  What data and features are typically used for AAPL                                                             │
│  - Price data: daily open, high, low, close, adjusted close, and volume. For robust performance, adjust for     │
│  splits and dividends, and align with market holidays.                                                          │
│  - Derived price features: daily returns, log returns, moving averages (e.g., 5D, 10D, 20D, 50D), rolling       │
│  standard deviation (volatility proxy), momentum indicators (e.g., RSI, MACD), and volatility-related signals   │
│  (e.g., range-based metrics, ATR proxies).                                                                      │
│  - Time-based features: day of week, month, quarter, and potential earnings-associated windows                  │
│  (pre/post-earnings days, though earning...                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Trading Strategy Developer                                                                              │
│                                                                                                                 │
│  Task: Develop potential trading strategies for AAPL that align with the user's medium risk tolerance and day   │
│  trading preferences.                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [ ]:
from IPython.display import Markdown

Markdown(results.raw)

NameError: name 'results' is not defined